# 1. Install and Import Required Packages

In [ ]:
!pip install xmltodict
!pip install split-folders
!pip install easyocr
!pip install GPUtil
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5 && pip install -r requirements.txt comet_ml  # install

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import cv2
import uuid
import time
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import xmltodict
import glob
import xml.etree.ElementTree as ET
import random as rnd
import splitfolders
import easyocr
import PIL
import copy

from pathlib import Path
from sklearn.model_selection import train_test_split
from collections import Counter
from PIL import Image
from tqdm.auto import tqdm
from GPUtil import showUtilization as gpu_usage
from numba import cuda
from timeit import default_timer as timer

from google.colab import files
from google.colab import drive
from google.colab.patches import cv2_imshow

import torch

import matplotlib
matplotlib.use('TkAgg')

# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
from matplotlib import patches as mpatches

In [ ]:
import comet_ml
import torch

# 2. Import a Dataset for Training

### Use only one of the follow datasets. I decided on option C for the best balance of training time, and accuracy

## Option A - 433 images (Requires Formating for use with Yolov5)

Souce: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection I added this to my Google Drive to quickly import in new session. Modify the following code snippet based on how you import the data

In [ ]:
drive.mount('/content/drive')
!cp -r drive/MyDrive/FYP/PlateRecognition .


define dictionary with the basic informations about the dataset.

In [ ]:
dataset = {
            "file":[],
            "width":[],
            "height":[],
            "xmin":[],
            "ymin":[],
            "xmax":[],
            "ymax":[]
           }

In [ ]:
img_names=[] 
annotations=[]
for dirname, _, filenames in os.walk("PlateRecognition"):
    for filename in filenames:
        if os.path.join(dirname, filename)[-3:]==("png" or "jpg"):
            img_names.append(filename)
        elif os.path.join(dirname, filename)[-3:]=="xml":
            annotations.append(filename)
    
img_names[:10]

In [ ]:
annotations[:10]

Rewrite the info from .xml to the dictionary. For each photo we can get multiple bonding boxes, therefore filenames, width and height will recur.

In [ ]:
path_annotations="PlateRecognition/annotations/*.xml" 

for item in glob.glob(path_annotations):
    tree = ET.parse(item)
    
    for elem in tree.iter():
        if 'filename' in elem.tag:
            filename=elem.text
        elif 'width' in elem.tag:
            width=int(elem.text)
        elif 'height' in elem.tag:
            height=int(elem.text)
        elif 'xmin' in elem.tag:
            xmin=int(elem.text)
        elif 'ymin' in elem.tag:
            ymin=int(elem.text)
        elif 'xmax' in elem.tag:
            xmax=int(elem.text)
        elif 'ymax' in elem.tag:
            ymax=int(elem.text)
            
            dataset['file'].append(filename)
            dataset['width'].append(width)
            dataset['height'].append(height)
            dataset['xmin'].append(xmin)
            dataset['ymin'].append(ymin)
            dataset['xmax'].append(xmax)
            dataset['ymax'].append(ymax)
        
classes = ['license']

YOLO model requires normalized data (in range 0 to 1) in format [class_id, x, y, width, height], where x, y are coordinates of the middle of the bounding box(with corresponding width and height). Data must be saved as a txt file with a name corresponding to an image.

In [ ]:
x_pos = []
y_pos = []
frame_width = []
frame_height = []

labels_path = Path("PlateRecognition/labels")

labels_path.mkdir(parents=True, exist_ok=True)

save_type = 'w'

for i, row in enumerate(df.iloc):
    current_filename = str(row.file[:-4])
    
    width, height, xmin, ymin, xmax, ymax = list(df.iloc[i][-6:])
    
    x=(xmin+xmax)/2/width
    y=(ymin+ymax)/2/height
    width=(xmax-xmin)/width
    height=(ymax-ymin)/height
    
    x_pos.append(x)
    y_pos.append(y)
    frame_width.append(width)
    frame_height.append(height)
    
    txt = '0' + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + '\n'
    
    if i > 0:
        previous_filename = str(df.file[i-1][:-4])
        save_type='a+' if current_filename == previous_filename else 'w'
    
    
    with open("PlateRecognition/labels/" + str(row.file[:-4]) +'.txt', save_type) as f:
        f.write(txt)
        
        
df['x_pos']=x_pos
df['y_pos']=y_pos
df['frame_width']=frame_width
df['frame_height']=frame_height

df

Use splitfolder library to split images and labels into training and validation sets

In [ ]:
input_folder = Path("PlateRecognition")
output_folder = Path("yolov5/data/PlateRecognition")
splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(0.8, 0.2),
    group_prefix=None
)
print("Moving files finished.")

Copying files: 1309 files [00:00, 2904.65 files/s]

Moving files finished.


Yolo requires config data in .yaml file. 

In [ ]:
import yaml

yaml_file = 'yolov5/data/plates.yaml'

yaml_data = dict(
    path = "data/PlateRecognition",
    train = "train",
    val = "val",
    nc = len(classes),
    names = classes
)

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f, explicit_start = True, default_flow_style = False)

## Option B - 21k images

Source: https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e/dataset/4 

In [ ]:
!curl -L "https://universe.roboflow.com/ds/2JyVmpjdM5?key=ynwxbq1iAt" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## Option C - 7K images

Source: https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e/dataset/2

In [ ]:
!curl -L "https://universe.roboflow.com/ds/xeLsSKTTwf?key=TDDMoeD2hz" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## *(Optional)* Clear the gpu memory & check device - Training will be extremely slow on CPU

In [ ]:
def free_gpu_cache() -> None:
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

In [ ]:
device = '0' if torch.cuda.is_available() else 'cpu' 
device

In [ ]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


# 3. Train Model
### For model installation and required downloads check this: https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

Comet API Keys for comparison of training runs (add own key)

In [ ]:
!export COMET_API_KEY=
!export COMET_PROJECT_NAME=yolov5 # This will default to 'yolov5'

Train the model:

In [ ]:
!cd yolov5 && python train.py --workers 2 --img 640 --batch 32 --epochs 100 --data "../data.yaml" --weights yolov5s.pt --device {device} --cache --save-period 1

Download Best Model (may need to change path to most recent run if testing different inputs in same session)

In [ ]:
!zip -r platemodel.zip yolov5/runs/train/exp5
files.download('platemodel.zip')

# 2. Test Model


Import Last Best Model - (if training session was lost/left)

In [ ]:
drive.mount('/content/drive')
!mkdir -p yolov5/runs/train/exp/weights
!cp drive/MyDrive/FYP/best.pt yolov5/runs/train/exp/weights/best.pt
# !cp drive/MyDrive/FYP/PlateTestVideo.mp4 .

Mounted at /content/drive


## Benchmark Model

In [ ]:
!cd yolov5 && python benchmarks.py --weights runs/train/exp/weights/best.pt --imgsz 640 --device 0

## Test on Photo


A photo must first uploaded in Colab, then change "test_phto_path" to match its path

In [ ]:
%matplotlib inline

test_photo_path = "Cars89.png"

results = model(test_photo_path)
detections=np.squeeze(results.render())

labels, coordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
image = cv2.imread(test_photo_path)
width, height = image.shape[1], image.shape[0]

print(f'Photo width,height: {width},{height}. Detected plates: {len(labels)}')

for i in range(len(labels)):
    row = coordinates[i]
    if row[4] >= 0.6:
        x1, y1, x2, y2 = int(row[0]*width), int(row[1]*height), int(row[2]*width), int(row[3]*height)
        plate_crop = image[int(y1):int(y2), int(x1):int(x2)]
        ocr_result = reader.readtext((plate_crop), paragraph="True", min_size=120, allowlist = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        # text=ocr_result[0][1]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 6) ## BBox
        # cv2.putText(image, f"{text}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 3)
        plt.axis(False)
        plt.imshow((image)[...,::-1])
        
        print(f'Detection: {i+1}. YOLOv5 prob: {row[4]:.2f}, easyOCR results: {ocr_result}')

Photo width,height: 400,262. Detected plates: 3
